In [6]:
import requests
import json
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.adapters import RequestsAdapter
import os
import certifi

# Sett opp SSL-sertifikat for å unngå feilmelding
os.environ["SSL_CERT_FILE"] = certifi.where()

# Bruk Geopy til å finne breddegrad og lengdegrad for en by
geolocator = Nominatim(user_agent="Dataanalyseapplikasjon")
location = geolocator.geocode(input("Hvilken by vil du ha værmelding for? Skriv på formen By,Land: "))


# Definer API-endepunkt og parametere
url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
params = {
    "lat": location.latitude,       # Breddegrad for utvalgt by
    "lon": location.longitude       # Lengdegrad for utvalgt by
}

# User-Agent er påkrevd av Yr
headers = {
    "User-Agent": "Miljoedataanalyseapplikasjon/0.1 (fredbm@stud.ntnu.no)"
}


# Gjør API-kallet
response = requests.get(url, headers=headers, params=params)

# Sjekk om forespørselen var vellykket
if response.status_code == 200:
    data = response.json()

    # Hent ut den nyeste værprognosen
    forecast = data["properties"]["timeseries"][0]
    time = forecast["time"]
    details = forecast["data"]["instant"]["details"]

    # Formater dataene
    temperature = details["air_temperature"]
    wind_speed = details["wind_speed"]
    precipitation = forecast["data"].get("next_1_hours", {}).get("details", {}).get("precipitation_amount", 0)

    # Konverter tidspunkt til leselig format
    time_formatted = datetime.fromisoformat(time.replace("Z", "+00:00")).strftime("%Y-%m-%d %H:%M:%S")

    # Print værdata på en ryddig måte
    print("=" * 40)
    print(f"📍 Værmelding for",location.address ,location.latitude, location.longitude)
    print(f"🕒 Tidspunkt: {time_formatted} UTC")
    print(f"🌡 Temperatur: {temperature}°C")
    print(f"💨 Vindhastighet: {wind_speed} m/s")
    print(f"🌧 Nedbør (neste time): {precipitation} mm")
    print("=" * 40)

else:
    print(f"Feil: {response.status_code} - {response.text}")


📍 Værmelding for Cape Town, City of Cape Town, Western Cape, 8001, South Africa -33.9288301 18.4172197
🕒 Tidspunkt: 2025-03-03 09:00:00 UTC
🌡 Temperatur: 23.3°C
💨 Vindhastighet: 4.2 m/s
🌧 Nedbør (neste time): 0.0 mm
